# Week 7: Web scraping, BS4, Selenium and Regexp
### Part 1
##### 30 min
1. Make a request to: `https://en.wikipedia.org/wiki/Alan_Turing` and print out the responses .text property
2. Find and print the title of the response page
3. Find and print content of the first p tag that has content.
4. Find and print all content from the TOC
##### 20 min
5. Create a dictionary from the TOC links like: {'first link':'#this_is_the_first_link}
### Pause
##### 40 min
6. Create a function that can take a word and look for it in the dictionary keys and then return the content from the first link that matches. Return the next p elements until next headline (h3 element).
7. Make it into a cli program.

### Pause

### Part 2
##### 40 min
1. Using regex find out how many times 'Turing' is used in the article
9. Regex: Find all the sentences that has a year in them (sentense defined by: starting at \n or dot or comma and ending at dot or comma.

### Part 3
##### 40 min
1. Use selenium to go to amazon.com and search for `Pet Shower Cap - Waterproof Reusable Bath Ear Covers`
2. Print how many products were found
3. Find the cheapest and the most expensive product from the list


Make a request to: `https://en.wikipedia.org/wiki/Alan_Turing` and print out the responses .text property

In [12]:
import requests 
import bs4
r = requests.get('https://en.wikipedia.org/wiki/Alan_Turing')
r.raise_for_status()
r.text[:500]
#soup = bs4.BeautifulSoup(r.text, 'html.parser')
#soup

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Alan Turing - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"56a6d37e-8797-46e3-9852-911b06caee1c","wg'

In [13]:
soup = bs4.BeautifulSoup(r.text, 'html.parser')

print(soup.select('title'))


[<title>Alan Turing - Wikipedia</title>]


Find and print content of the first p tag that has content.

In [14]:
p_elems = soup.select('p')

for p in p_elems:
    if len(p.getText().strip()) > 0:
        print(p.getText())
        break
        
        

Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologist.[6] Turing was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer.[7][8][9] He is widely considered to be the father of theoretical computer science and artificial intelligence.[10]



Find and print all content from the TOC

In [15]:
toc = soup.select('.toc > ul > li')

In [16]:
for el in toc:
    print(el.getText())

1 Early life and education

1.1 Family
1.2 School
1.3 Christopher Morcom
1.4 University and work on computability


2 Career and research

2.1 Cryptanalysis
2.2 Bombe
2.3 Hut 8 and the naval Enigma
2.4 Turingery
2.5 Delilah
2.6 Early computers and the Turing test
2.7 Pattern formation and mathematical biology


3 Personal life

3.1 Engagement
3.2 Homosexuality and indecency conviction
3.3 Treasure
3.4 Death
3.5 Government apology and pardon


4 Legacy

4.1 Awards, honours, and tributes
4.2 Centenary celebrations


5 Notes and references

5.1 Notes
5.2 References
5.3 Sources


6 Further reading

6.1 Articles
6.2 Books


7 External links


Create a dictionary from the TOC links like: {'first link':'#this_is_the_first_link}

In [17]:
outer_toc = soup.select('.toc > ul > li')
inner_toc = soup.select('.toc > ul > li > ul > li')

toc_dict={}
#for el in outer_toc:
    #text = el.getText()
    #a = el.select('a')
    #val = a[0].get('href')
    #key = text.split(" ",1)[1]
    #key = key.split("\n")[0]
    #toc_dict[key]=val
    
for el in inner_toc:
    text = el.getText()
    a = el.select('a')
    val = a[0].get('href')
    key = text.split(" ",1)[1]
    key = key.split("\n")[0].lower()
    toc_dict[key]=val
toc_dict

{'family': '#Family',
 'school': '#School',
 'christopher morcom': '#Christopher_Morcom',
 'university and work on computability': '#University_and_work_on_computability',
 'cryptanalysis': '#Cryptanalysis',
 'bombe': '#Bombe',
 'hut 8 and the naval enigma': '#Hut_8_and_the_naval_Enigma',
 'turingery': '#Turingery',
 'delilah': '#Delilah',
 'early computers and the turing test': '#Early_computers_and_the_Turing_test',
 'pattern formation and mathematical biology': '#Pattern_formation_and_mathematical_biology',
 'engagement': '#Engagement',
 'homosexuality and indecency conviction': '#Homosexuality_and_indecency_conviction',
 'treasure': '#Treasure',
 'death': '#Death',
 'government apology and pardon': '#Government_apology_and_pardon',
 'awards, honours, and tributes': '#Awards,_honours,_and_tributes',
 'centenary celebrations': '#Centenary_celebrations',
 'notes': '#Notes',
 'references': '#References',
 'sources': '#Sources',
 'articles': '#Articles',
 'books': '#Books'}

Create a function that can take a word and look for it in the dictionary keys and then return the content from the first link that matches. Return the next p elements until next headline (h3 element).

In [24]:
def look_up(word):
    wordlist = list(toc_dict.keys())
    
    if word.lower() in wordlist:
        id_of_el = toc_dict[word.lower()]
        
        r = requests.get('https://en.wikipedia.org/wiki/Alan_Turing')
        r.raise_for_status()
        soup = bs4.BeautifulSoup(r.text, 'html.parser')
        tags = soup.find_all(['p', 'h3','span'])
        section_found = False
        for tag in tags:
            if tag.get('id') != None and tag.name == 'span':
                if tag.get('id') in id_of_el:
                    section_found = True
            if section_found == True:
                if tag.name == 'p':
                    print(tag.text)
                if tag.name == 'h3':
                    break;
        
    else:
        return 'Word not found'


In [29]:
look_up('death')

On 8 June 1954, at his house at 43 Adlington Road, Wilmslow,[153] Turing's housekeeper found him dead. He had died the previous day at the age of 41. Cyanide poisoning was established as the cause of death.[154] When his body was discovered, an apple lay half-eaten beside his bed, and although the apple was not tested for cyanide,[155] it was speculated that this was the means by which Turing had consumed a fatal dose. An inquest determined that he had committed suicide.[145] Andrew Hodges and another biographer, David Leavitt, have both speculated that Turing was re-enacting a scene from the Walt Disney film Snow White and the Seven Dwarfs (1937), his favourite fairy tale. Both men noted that (in Leavitt's words) he took "an especially keen pleasure in the scene where the Wicked Queen immerses her apple in the poisonous brew".[156] Turing's remains were cremated at Woking Crematorium on 12 June 1954,[157] and his ashes were scattered in the gardens of the crematorium, just as his fath

Using regex find out how many times 'Turing' is used in the article

In [202]:
r = requests.get('https://en.wikipedia.org/wiki/Alan_Turing')
r.raise_for_status()
soup = bs4.BeautifulSoup(r.text, 'html.parser')

import re
turing_reg = re.compile(r'Turing')
len(turing_reg.findall(soup.text))

438

Regex: Find all the sentences that has a year in them (sentense defined by: starting at \n or dot or comma and ending at dot or comma.

In [219]:


sentences_reg = re.compile(r'[^.\n]* \d{4}[$.\n]*.')
sentences_reg.findall(soup.text)

['\xa01928 at age 16Born(1912-06-23)23 June 1912Maida Vale, London, EnglandDied7 June 1954(',
 'Partner(s)Joan Clarke(engaged in 1941;',
 'Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912\xa0– 7 June 1954)',
 ' In 1938,',
 ' In 1948,',
 ' He wrote a paper on the chemical basis of morphogenesis[1] and predicted oscillating chemical reactions such as the Belousov–Zhabotinsky reaction, first observed in the 1960s',
 'Turing was prosecuted in 1952 ',
 ' Turing died on 7 June 1954,',
 'Following a public campaign in 2009,',
 ' Queen Elizabeth II granted a posthumous pardon in 2013. ',
 'The term "Alan Turing law" is now used informally to refer to a 2017 ',
 ' He appears on the current Bank of England £50 note, which was released on 23 June 2021,',
 ' A 2019 ',
 " Turing's mother, Julius's wife, was Ethel Sara Turing (née\xa0Stoney; 1881–",
 '[18] Julius and Ethel married on 1 Oct 1907 ',
 ' However, both Julius and Ethel wanted their children to be brought up in Britain, so they move

1. Use selenium to go to amazon.com and search for Pet Shower Cap - Waterproof Reusable Bath Ear Covers
2. Print how many products were found
3. Find the cheapest and the most expensive product from the list

In [1]:
# Example: goto www.cphbusiness.dk and find all the "Erhvervsakademiuddannelser" that are available.

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By


import bs4
import json

url = 'https://www.cphbusiness.dk'
def cphbusiness_interaction():
    #profile = webdriver.FirefoxProfile()
    #profile.set_preference("general.useragent.override", "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0")
    
    # headless is needed here because we do not have a GUI version of firefox
    options = Options()
    options.headless = True
    browser = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=options)
    
    browser.get(url)
    browser.implicitly_wait(2)
    button = browser.find_element(By.ID,'declineButton')
    button.click()
    
    button = browser.find_element(By.XPATH,'/html/body/header/div[3]/div[4]/div/nav/ul/li[1]/a')
    button.click()
    
    button = browser.find_element('xpath','/html/body/main/div[1]/div/div[1]/div/a') # using xpath string
    button.click()
    
    edu_buttons = browser.find_elements('css selector','div.tile.tile--small.small-12.medium-12.large-6.columns')
    edu_buttons = browser.find_elements('css selector','p.tile__link.tile__link--small a.icon-arrow-after')
    educations = [b.text for b in edu_buttons]
    return educations, browser.page_source
    
def find_elements(page, selector):
    soup = bs4.BeautifulSoup(page, 'html.parser')
    event_cells = soup.select(selector)
    return event_cells

def print_page(page,file):
    with open(file,'w') as file:
        file.write(json.dumps(page))

        

In [2]:
educations,source = cphbusiness_interaction()
print(educations)


[WDM] - Downloading: 16.2kB [00:00, 14.4MB/s]                   
[WDM] - Downloading: 100%|██████████| 2.59M/2.59M [00:00<00:00, 13.6MB/s]


WebDriverException: Message: Process unexpectedly closed with status 1


In [7]:
from selenium import webdriver
import geckodriver_autoinstaller


geckodriver_autoinstaller.install()  # Check if the current version of geckodriver exists
                                     # and if it doesn't exist, download it automatically,
                                     # then add geckodriver to path

driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title


Command '/usr/bin/firefox' requires the firefox snap to be installed.
Please install it with:

snap install firefox



WebDriverException: Message: Process unexpectedly closed with status 1


In [8]:
%%bash
pip install webdriver_manager